In [3]:
from feature.clip_base_dataset import *
from feature.after_dataset import *
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [4]:
train_input_list = sorted(glob('./data/train_input/*.csv'))
train_target_list = sorted(glob('./data/train_target/*.csv'))

test_input_list = sorted(glob('./data/test_input/*.csv'))
test_target_list = sorted(glob('./data/test_target/*.csv'))

In [5]:
def group_statistics(df):  
    '''
    변수의 Case별로 DAT을 기준으로 groupby 하여 대표값 설정  
    mean, median, std, var, quantile
    '''
    df_a = pd.DataFrame()

    for i, v in enumerate(df["Case"].unique()):
        train_old = df[df['Case']==v]
        train_old = train_old.groupby(['DAT']).sem().reset_index()
        df_a = pd.concat([df_a, train_old])
        df_a = df_a.drop(['일간누적분무량', '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량',
                          'predicted_weight_g', 'Case', 'DAT', 'obs_time'], axis=1)
    return df_a

In [6]:
train, train2 = make_dataset(train_input_list, train_target_list)
test, test2 = make_dataset(test_input_list, test_target_list)

train['obs_time'] = train2['obs_time']
train['DAT'] = train2['DAT']
test['obs_time'] = test2['obs_time']
test['DAT'] = test2['DAT']

train = time_value(train)
test = time_value(test)

train = limit_range(train)
test = limit_range(test)

train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

test['predicted_weight_g'] = 0

train = col_cumsum(train, "시간당분무량", "일간누적분무량")
train = col_cumsum(train, "시간당백색광량", "일간누적백색광량")
train = col_cumsum(train, "시간당적색광량", "일간누적적색광량")
train = col_cumsum(train, "시간당청색광량", "일간누적청색광량")
train = col_cumsum(train, "시간당총광량", "일간누적총광량")

In [7]:
train = group_statistics(train)
test = group_statistics(test)

In [8]:
train

,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,시간당백색광량,시간당적색광량,시간당청색광량,시간당총광량
0,0.264396,0.593556,5.417703,0.002284,20.638324,1865.030305,99.787974,0.000000,1964.815637
1,0.290969,0.588704,5.260259,0.001647,20.704571,1863.969268,99.694257,0.000000,1963.663523
2,0.175107,0.618549,3.254476,0.000809,20.916494,1861.879614,99.567537,0.000000,1961.447133
3,0.155099,0.606251,26.648961,0.000563,20.704571,1863.969268,99.694257,0.000000,1963.663523
4,0.204568,0.575020,4.129329,0.001164,21.879091,1862.723667,99.619655,0.000000,1962.343321
...,...,...,...,...,...,...,...,...,...
23,0.557887,0.244215,3.768624,0.006900,143.249578,763.282191,116.495251,55.001939,934.747661
24,0.280801,0.264483,4.896206,0.000626,21.194461,1259.730451,192.469235,91.094327,1543.262729
25,0.189144,0.198145,10.644756,0.000620,17.298506,1257.858096,192.192321,90.963292,1540.976752
26,0.254904,0.160352,5.432403,0.002546,17.361761,1259.794961,192.539295,91.156533,1543.427248


In [9]:
train.to_csv('clip_train_sem.csv', index=False)
test.to_csv('clip_test_sem.csv', index=False)